In [11]:
from google.colab import drive 
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [12]:
!cp -av 'drive/MyDrive/CV/final_project/data_pure.zip' './' 
!unzip -u -q "data_pure.zip"

'drive/MyDrive/CV/final_project/data_pure.zip' -> './data_pure.zip'


In [18]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
seed = 1
base_dir = './'
df = pd.read_csv(base_dir+"data/train.csv")
df['filename'] = df.apply(lambda row: row['id']+'.jpg',axis=1)

def prep_fn(img):
    img = img.astype(np.float32) / 255.0
    img = (img - 0.5) * 2
    return img

data_gen_args = dict(preprocessing_function=prep_fn,
                     width_shift_range=0.2,
                     height_shift_range=0.05,
                     zoom_range=0.05,
                     rotation_range=5,
                     horizontal_flip=False,
                     vertical_flip=False,
                     validation_split=0.1)

train_datagen = ImageDataGenerator(**data_gen_args)
val_datagen = ImageDataGenerator(preprocessing_function=prep_fn,validation_split=0.1)

train_generator = train_datagen.flow_from_dataframe(dataframe=df,
                                                    directory=base_dir+'data/train/',
                                                    x_col='filename',
                                                    y_col=['x','y'],
                                                    subset="training",
                                                    batch_size = 32,
                                                    seed=seed,
                                                    shuffle=True,
                                                    class_mode='raw',
                                                    target_size=(224,224))
val_generator = val_datagen.flow_from_dataframe(dataframe=df,
                                                directory=base_dir+'data/train/',
                                                x_col='filename',
                                                y_col=['x','y'],
                                                subset="validation",
                                                batch_size = 32,
                                                seed=seed,
                                                shuffle=True,
                                                class_mode='raw',
                                                target_size=(224,224))

Found 6750 validated image filenames.
Found 750 validated image filenames.


In [ ]:
df_test = pd.read_csv("data/imagenames.csv")
df_test['filename'] = df_test.apply(lambda row: row['id']+'.jpg',axis=1)
df_test['x']=0
df_test['y']=0
test_datagen = ImageDataGenerator(preprocessing_function=prep_fn)
test_generator = test_datagen.flow_from_dataframe(dataframe=df_test,
                                                directory=base_dir+'data/test/',
                                                x_col='filename',
                                                y_col=['x','y'],
                                                batch_size = 32,
                                                shuffle=False,
                                                class_mode='raw',
                                                target_size=(224,224))

Found 1200 validated image filenames.


In [1]:
from tensorflow.keras.applications import VGG16,Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D,Flatten, Dense, Dropout

In [7]:
def regressor_v1(input_size = (224,224,3)):
    img_input = Input(shape=input_size)
    x = Conv2D(32, (5, 5), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(32, (5, 5), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2pool')(x)
    
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    x = Flatten()(x)
    x = Dense(512,activation='relu')(x)
    x = Dense(2)(x)
    model = Model(img_input,x)
    model.compile(optimizer='Adam',loss=MeanAbsoluteError())
    return model

In [15]:
model = regressor_v1()
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 32)      2432      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 32)      25632     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 32)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 64)      18496     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 64)      36928     
_________________________________________________________________
block2pool (MaxPooling2D)    (None, 56, 56, 64)        0   

In [19]:
with tf.device('/device:GPU:0'):
    #model.load_weights('drive/MyDrive/CV/Model/regressor.h5')
    #model.trainable = True
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
    r = model.fit(train_generator, 
                  validation_data=val_generator,
                  validation_steps=20,
                  steps_per_epoch=211,
                  epochs=100, 
                  callbacks=[callback])

Epoch 1/100
211/211 [==============================] - 99s 471ms/step - loss: 23.1552 - val_loss: 24.3014
Epoch 2/100
211/211 [==============================] - 99s 470ms/step - loss: 22.5882 - val_loss: 23.6225
Epoch 3/100
211/211 [==============================] - 99s 470ms/step - loss: 22.0246 - val_loss: 24.6145
Epoch 4/100
211/211 [==============================] - 99s 471ms/step - loss: 21.6082 - val_loss: 23.5454
Epoch 5/100
211/211 [==============================] - 99s 471ms/step - loss: 20.9491 - val_loss: 21.9757
Epoch 6/100
211/211 [==============================] - 99s 471ms/step - loss: 20.3880 - val_loss: 21.8728
Epoch 7/100
211/211 [==============================] - 99s 470ms/step - loss: 19.6885 - val_loss: 21.8454
Epoch 8/100
211/211 [==============================] - 99s 469ms/step - loss: 19.4269 - val_loss: 21.5493
Epoch 9/100
211/211 [==============================] - 99s 469ms/step - loss: 18.8019 - val_loss: 22.3931
Epoch 10/100
211/211 [========================

In [20]:
def location_regressor():
    Xception_model = Xception(input_shape=(224,224,3),include_top=False,weights='imagenet')
    for layer in Xception_model.layers:
        layer.trainable = False
    x = Flatten()(Xception_model.output)
    #x = GlobalAveragePooling2D()(Xception_model.output)
    x=Dense(1024,activation='relu')(x)
    x=Dense(128,activation='relu')(x)
    x=Dense(2)(x)
    model = Model(inputs=Xception_model.input, outputs=x)
    model.compile(optimizer='Adam',loss=MeanAbsoluteError())
    return model

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [22]:
with tf.device('/device:GPU:0'):
    model = location_regressor()
    model.load_weights('drive/MyDrive/CV/Model/regressor.h5')
    #model.trainable = True
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    r=model.fit(train_generator, validation_data=val_generator, 
                validation_steps=20, steps_per_epoch=211,epochs=100,callbacks=[callback])

Epoch 1/100
211/211 [==============================] - 104s 483ms/step - loss: 9.2005 - val_loss: 15.1464
Epoch 2/100
211/211 [==============================] - 101s 479ms/step - loss: 8.7609 - val_loss: 14.8011
Epoch 3/100
211/211 [==============================] - 101s 477ms/step - loss: 8.5541 - val_loss: 15.2953
Epoch 4/100
211/211 [==============================] - 101s 477ms/step - loss: 8.4219 - val_loss: 14.6739
Epoch 5/100
211/211 [==============================] - 100s 476ms/step - loss: 8.0346 - val_loss: 14.8811
Epoch 6/100
211/211 [==============================] - 101s 476ms/step - loss: 7.8682 - val_loss: 14.5456
Epoch 7/100
211/211 [==============================] - 101s 477ms/step - loss: 7.7502 - val_loss: 15.3763
Epoch 8/100
211/211 [==============================] - 101s 477ms/step - loss: 7.8012 - val_loss: 14.5588


In [ ]:
model.save('drive/MyDrive/CV/Model/regressor.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
with tf.device('/device:GPU:0'):
    ypred = model.predict(test_generator)
ypred.shape

(1200, 2)

In [ ]:
df_test['x'] = ypred[:,0]
df_test['y'] = ypred[:,1]
if 'filename' in df_test.keys():
    df_test=df_test.drop(columns=['filename'])
df_test.head()
df_test.to_csv('pred_regression.csv',index=False)